In [1]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
import math
import random
import unidecode
import datetime
from time import strptime

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinatti',
                        'pirates': 'Pittsburg',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinatti',
                        'pirates': 'Pittsburg',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinatti': 'NL',
          'Pittsburg': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']

In [59]:
class Batter_Improved:
    """Class that holds information for a Batter.
    
    Attributes:
        team: team name of player
        name: player name
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, singles_R, doubles_R, triples_R,
                 home_runs_R, walks_R, sb_rate, sb_success):
        """Initializes values for this class"""
        
        self.team = team
        self.name = name
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = singles_L + doubles_L
        self.triples_L = singles_L + doubles_L + triples_L
        self.home_runs_L = singles_L + doubles_L + triples_L + home_runs_L
        self.walks_L = singles_L + doubles_L + triples_L + home_runs_L + walks_L
        self.singles_R = singles_R
        self.doubles_R = singles_R + doubles_R
        self.triples_R = singles_R + doubles_R + triples_R
        self.home_runs_R = singles_R + doubles_R + triples_R + home_runs_R
        self.walks_R = singles_R + doubles_R + triples_R + home_runs_R + walks_R
        self.sb_rate = sb_rate
        self.sb_success = sb_success
        self.cum_singles_L = 0
        self.cum_doubles_L = 0
        self.cum_triples_L = 0
        self.cum_home_runs_L = 0
        self.cum_walks_L = 0
        self.cum_pa_L = 0
        self.cum_singles_R = 0
        self.cum_doubles_R = 0
        self.cum_triples_R = 0
        self.cum_home_runs_R = 0
        self.cum_walks_R = 0
        self.cum_pa_R = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
    
    def cum_pa(self, hand):
        """cumulative plate appearances value of a batter for particular pitcher handedness
        
        Returns:
            float: value for cumulative plate appearances
        """
        if(hand=='L'):
            return self.cum_pa_L
        elif(hand=='R'):
            return self.cum_pa_R
    
    def singles(self, hand):
        """singles value of a batter for particular pitcher handedness
        
        Returns:
            float: value for singles
        """
        if(hand=='L'):
            return self.singles_L
        elif(hand=='R'):
            return self.singles_R

    def doubles(self, hand):
        """doubles value of a batter for particular pitcher handedness
        
        Returns:
            float: value for doubles
        """
        if(hand=='L'):
            return self.doubles_L
        elif(hand=='R'):
            return self.doubles_R
        
    def triples(self, hand):
        """triples value of a batter for particular pitcher handedness
        
        Returns:
            float: value for triples
        """
        if(hand=='L'):
            return self.triples_L
        elif(hand=='R'):
            return self.triples_R

    def home_runs(self, hand):
        """home runs value of a batter for particular pitcher handedness
        
        Returns:
            float: value for home runs
        """
        if(hand=='L'):
            return self.home_runs_L
        elif(hand=='R'):
            return self.home_runs_R
        
    def walks(self, hand):
        """walks value of a batter for particular pitcher handedness
        
        Returns:
            float: value for walks
        """
        if(hand=='L'):
            return self.walks_L
        elif(hand=='R'):
            return self.walks_R
    
    def cum_singles(self, hand):
        """cum_singles of a batter for particular pitcher handedness
        
        Returns:
            float: value for cum_singles
        """
        if(hand=='L'):
            return self.cum_singles_L
        elif(hand=='R'):
            return self.cum_singles_R

    def cum_doubles(self, hand):
        """cum_doubles of a batter for particular pitcher handedness
        
        Returns:
            float: value for cum_doubles
        """
        if(hand=='L'):
            return self.cum_doubles_L
        elif(hand=='R'):
            return self.cum_doubles_R
        
    def cum_triples(self, hand):
        """cum_triples of a batter for particular pitcher handedness
        
        Returns:
            float: value for cum_triples
        """
        if(hand=='L'):
            return self.cum_triples_L
        elif(hand=='R'):
            return self.cum_triples_R

    def cum_home_runs(self, hand):
        """cum_home_runs of a batter for particular pitcher handedness
        
        Returns:
            float: value for cum_home_runs
        """
        if(hand=='L'):
            return self.cum_home_runs_L
        elif(hand=='R'):
            return self.cum_home_runs_R
        
    def cum_walks(self, hand):
        """cum_walks of a batter for particular pitcher handedness
        
        Returns:
            float: value for cum_walks
        """
        if(hand=='L'):
            return self.cum_walks_L
        elif(hand=='R'):
            return self.cum_walks_R
    
    
    def hits(self):
        """total hits of a batter
        
        Returns:
            int: number of hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form"""
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form"""
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg() + self.obp())
    
    def print_stats(self):
        """Prints some stats of a player"""

        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """total hits of a batter against lefties
        
        Returns:
            int: number of hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form"""
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """total hits of a batter against righties
        
        Returns:
            int: number of hits
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form"""
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher_Improved(Batter_Improved):
    """Class that holds information for a pitcher.
    
    Attributes:
        team: team name of player
        name: player name
        position: player position
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, singles_R, doubles_R, 
                 triples_R, home_runs_R, walks_R, sb_rate, sb_success):
        """Initializes values for this class"""
        
        Batter_Improved.__init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, 
                                 singles_R, doubles_R, triples_R, home_runs_R, walks_R, sb_rate, sb_success)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
    
    def cum_outs(self, hand):
        """cumulative outs of a batter for particular batter handedness
        
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)"""
        ip = (self.cum_outs_L+self.cum_outs_R)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form"""
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form"""
        if(self.cum_outs==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walk_R)+self.hits())/((self.cum_outs_L+self.cum_outs_R)/3)), 2))

class Team_Improved:
    """Class that holds information for a team.
    
    Attributes:
        city: team city
        roster: batting lineup of team in list form
        batting_index: position in batting order of current batter
        rotation_index: position in rotation of current pitcher
        runners: list representing whether a runner is on each base
        runs: number of runs fr a team in a game
        wins: number of wins for a team
        losses: number of losses for a team
        cumulative runs: total runs for a team over a span of games
    """
    def __init__(self, city, league, batter_df, pitcher_df, starters, relievers):
        """Initializes values for this class"""
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.pitcher_df = pitcher_df
        self.starters = starters
        self.relievers = relievers
        self.pitching_staff = pitcher_df
        self.closer = None
        self.current_pitcher_list = []
        self.available_pitcher_list = []
        self.current_pitcher = None
        self.batting_index = 0
        self.rotation_index = 0
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        self.pitchers = []
        self.batters = []
        self.game_no = 0
        
        for index, batter in batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            singles_L = round(batter['1B_L']/batter['PA_L'], 3)
            doubles_L = round(batter['2B_L']/batter['PA_L'], 3)
            triples_L = round(batter['3B_L']/batter['PA_L'], 3)
            home_runs_L = round(batter['HR_L']/batter['PA_L'], 3)
            walks_L = round(batter['BB_L']/batter['PA_L'], 3)
            singles_R = round(batter['1B_R']/batter['PA_R'], 3)
            doubles_R = round(batter['2B_R']/batter['PA_R'], 3)
            triples_R = round(batter['3B_R']/batter['PA_R'], 3)
            home_runs_R = round(batter['HR_R']/batter['PA_R'], 3)
            walks_R = round(batter['BB_R']/batter['PA_R'], 3)
            
            sb_rate = round((batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R'])/
                            (batter['1B_L']+batter['1B_R']+batter['BB_L']+batter['BB_R']), 3)
            if(batter['CS_L']+batter['CS_R'])==0:
                sb_success = 0
            else:
                sb_success = (batter['SB_L']+batter['SB_R'])/(batter['CS_L']+batter['CS_R'])

            self.batters.append(Batter_Improved(self.city, batter['Name'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                                home_runs_L, walks_L, singles_R, doubles_R, triples_R, home_runs_R, 
                                                walks_R, sb_rate, sb_success))

        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            
            
            ip_L = math.modf(pitcher['IP_L'])
            outs_L = int(ip_L[0]*10 + ip_L[1]*3)
            num_singles_L = pitcher['H_L']-pitcher['2B_L']-pitcher['3B_L']-pitcher['HR_L']
            singles_L = round(num_singles_L/pitcher['TBF_L'], 3)
            doubles_L = round(pitcher['2B_L']/pitcher['TBF_L'], 3)
            triples_L = round(pitcher['3B_L']/pitcher['TBF_L'], 3)
            home_runs_L = round(pitcher['HR_L']/pitcher['TBF_L'], 3)
            walks_L = round(pitcher['BB_L']/pitcher['TBF_L'], 3)
            
            ip_R = math.modf(pitcher['IP_R'])
            outs_R = int(ip_R[0]*10 + ip_R[1]*3)
            num_singles_R = pitcher['H_R']-pitcher['2B_R']-pitcher['3B_R']-pitcher['HR_R']
            singles_R = round(num_singles_R/pitcher['TBF_R'], 3)
            doubles_R = round(pitcher['2B_R']/pitcher['TBF_R'], 3)
            triples_R = round(pitcher['3B_R']/pitcher['TBF_R'], 3)
            home_runs_R = round(pitcher['HR_R']/pitcher['TBF_R'], 3)
            walks_R = round(pitcher['BB_R']/pitcher['TBF_R'], 3)
            
            new_pitcher = Pitcher_Improved(self.city, pitcher['Name'], pitcher['Throws'], singles_L, doubles_L, 
                                                  triples_L, home_runs_L, walks_L, singles_R, doubles_R, triples_R, 
                                                  home_runs_R, walks_R, 0, 0)
            
            self.pitchers.append(new_pitcher)
            
            if(new_pitcher.name==starters['closer']):
                self.closer = new_pitcher

        def make_lineup(batters, starters, league):
            lineup = []
            
            for starter in starters['lineup']:
                for batter in batters:
                    if(batter.name==starter):
                        lineup.append(batter)
                        
            if(league=='NL'):
                while(len(lineup)<8):
                    lineup.append(Batter_Improved(self.city, 'extra' +str(len(lineup)), 'R', .15, .05, .02, .03, .04, .15, .05,
                                         .02, .03, .04,.02, .01))
                
                lineup.append(Batter_Improved(self.city, 'pitcher', 'R', .08, .02, .005, .01, .04, .08, .02, .005, .01, .04, 0, 0))
            elif(league=='AL'):
                while(len(lineup)<9):
                    lineup.append(Batter_Improved(self.city, 'extra' +str(len(lineup)), 'R', .15, .05, .02, .03, .04, .15, .05, 
                                         .02, .03, .04, .02, .01))             
                
            return lineup
        
        def make_rotation(pitchers, starters):
            rotation = []
            
            for starter in starters['rotation']:
                for pitcher in pitchers:
                    if(pitcher.name==starter):
                        rotation.append(pitcher)
                        
            while(len(rotation)<5):
                rotation.append(Pitcher_Improved(self.city, 'extra'+ str(len(rotation)), 'R', .16, .06, .02, .03, 
                                                 .05, .16, .06, .02, .03, .05, 0, 0))
                
            return rotation
        
        def make_bullpen(pitchers, relievers):
    
            l_reliever_dict = {}
            l_innings_total = 0
            r_reliever_dict = {}
            r_innings_total = 0
            for index, reliever in relievers.iterrows():
                for pitcher in pitchers:
                    if(reliever['Name']==pitcher.name and reliever['Name']!=starters['closer']):
                        if(pitcher.hand=='L'):
                            l_reliever_dict.update({pitcher: float(reliever['IP'])})
                            l_innings_total+=float(reliever['IP'])
                        elif(pitcher.hand=='R'):
                            r_reliever_dict.update({pitcher: float(reliever['IP'])})
                            r_innings_total+=float(reliever['IP'])
                        
            relief_df = pd.DataFrame(columns=['Pitcher', 'value'])
            l_value = 0
            for pitcher, ip in l_reliever_dict.items():
                l_value+=(ip/l_innings_total)
                relief_df = relief_df.append({'Pitcher': pitcher, 'value': l_value}, ignore_index=True)

            r_value = 1
            for pitcher, ip in r_reliever_dict.items():
                r_value+=(ip/r_innings_total)
                relief_df = relief_df.append({'Pitcher': pitcher, 'value': r_value}, ignore_index=True)
                
                
            return relief_df
            
            
        self.lineup = make_lineup(self.batters, self.starters, self.league)
        self.rotation = make_rotation(self.pitchers, self.starters)
        self.bullpen_df = make_bullpen(self.pitchers, self.relievers)
        
    def choose_pitcher(self, outs, inning, runs, opp_runs, lineup, lineup_index):
        "Chooses pitcher to pitch for a team"

        self.available_pitchers = []
        for pitcher in self.pitchers:
            if pitcher not in self.current_pitcher_list:
                self.available_pitchers.append(pitcher)
            
        
        if(self.current_pitcher.current_bf<3 and self.current_pitcher.finished_half==False):
            return self.current_pitcher
        

        total = ((self.current_pitcher.current_bf-self.current_pitcher.current_outs)*3 + 
                 self.current_pitcher.current_outs + self.current_pitcher.current_runs*3)

        if(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3.0>0):
            total+=(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3)*2

        if(pitcher.hand==lineup[lineup_index].hand):
            total-=3
        
        if(total<=50 and self.current_pitcher in self.rotation or total<=7 and self.current_pitcher not 
           in self.rotation):
            return self.current_pitcher
        else:                
            if(inning==8 and outs>0 and self.closer not in self.current_pitcher_list 
               and runs-opp_runs>0 and runs-opp_runs<3):
                self.current_pitcher = self.closer
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            elif(inning>8 and self.closer not in self.current_pitcher_list and runs-opp_runs>0 and runs-opp_runs<4):
                self.current_pitcher = self.closer
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                left_count = 0
                right_count = 0

                for num in range(3):
                    if(lineup[(lineup_index+num)%9].hand=='L'):
                        left_count+=1
                    elif(lineup[(lineup_index+num)%9].hand=='R'):
                        right_count+=1

                if(left_count>right_count):
                    hand_indicator = 0
                else:
                    hand_indicator = 1

                reliever_available = False
                lefties_available = 0
                righties_available = 0
                for index, row in self.bullpen_df.iterrows():
                    if(row['Pitcher'] in self.available_pitchers):
                        reliever_available = True

                        if(row['Pitcher'].hand=='L'):
                            lefties_available+=1
                        elif(row['Pitcher'].hand=='R'):
                            righties_available+=1

                if(reliever_available):
                    if(lefties_available>0 and hand_indicator==0 or righties_available>0 and hand_indicator==1):
                        attempts = 0

                        while(attempts<100):
                            num = random.uniform(0+hand_indicator, 1+hand_indicator)

                            for index, row in self.bullpen_df.iterrows():
                                if(num<row['value'] and row['Pitcher'] not in self.current_pitcher_list):
                                    print('found pitcher')
                                    self.current_pitcher = row['Pitcher']
                                    self.current_pitcher_list.append(self.current_pitcher)
                                    return self.current_pitcher

                            attempts+=1
                    hand_indicator = -(hand_indicator-1)      


                    if(lefties_available>0 or righties_available>0):
                        attempts = 0

                        while(attempts<100):
                            num = random.uniform(0+hand_indicator, 1+hand_indicator)

                            for index, row in self.bullpen_df.iterrows():
                                if(num<row['value'] and row['Pitcher'] not in self.current_pitcher_list):
                                    self.current_pitcher = row['Pitcher']
                                    self.current_pitcher_list.append(self.current_pitcher)
                                    return self.current_pitcher

                            attempts+=1
                    else:
                        if(self.closer in self.available_pitchers):
                            self.current_pitcher = self.closer
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                        else:
                            num = random.randint(0, len(self.available_pitchers)-1)
                            self.current_pitcher = self.available_pitchers[num]
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                        
                
        print("HI")                                                        
        return self.current_pitcher
                
                
                
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started"""
        for pitcher in self.current_pitcher_list:
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
            
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
    def reset_runners(self):
        """Resets baserunners after each half inning"""
        self.runners = [None, None, None, 0]
    
    def cum_stats_df(self):
        """Class that holds information for a Batter.
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats"""
        
        batter_df = pd.DataFrame(columns=['Name', 'PA', '1B', '2B', '3B', 'HR', 'BB', 'AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_pa, b.cum_singles, b.cum_doubles, b.cum_triples, b.cum_home_runs, b.cum_walks,
                             b.avg(), b.obp(), b.slg(), b.ops(), b.cum_runs, b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
     
        pitcher_df = pd.DataFrame(columns=['Name', 'IP', 'H', 'BB', 'ERA', 'WHIP', 'HR','AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'SB', 'CS'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.ip(), p.hits(), p.cum_walks, p.era(), p.whip(), p.cum_home_runs, p.avg(),
                             p.obp(), p.slg(), p.ops(), p.cum_runs, p.cum_sb, p.cum_cs]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
            
        return batter_df, pitcher_df
    
class Game_Improved:
    """Class that holds information for a Game.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team
                                
    def get_stolen_base_result(self, runners, pitcher, num):
        
        if(runners[0]!=None and runners[1]==None):
            if(num>runners[0].sb_rate):
                return None
            else:
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        else:
            return None
        
    def get_result(self, batter, pitcher, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        pitcher.current_bf+=1
        
        batter_cum_pa = batter.cum_pa(batter_against)      
        batter_cum_singles = batter.cum_singles(batter_against)
        batter_cum_doubles = batter.cum_doubles(batter_against)      
        batter_cum_triples = batter.cum_triples(batter_against)
        batter_cum_home_runs = batter.cum_home_runs(batter_against)      
        batter_cum_walks = batter.cum_walks(batter_against)
        pitcher_cum_pa = pitcher.cum_pa(pitcher_against)      
        pitcher_cum_singles = pitcher.cum_singles(pitcher_against)
        pitcher_cum_doubles = pitcher.cum_doubles(pitcher_against)      
        pitcher_cum_triples = pitcher.cum_triples(pitcher_against)
        pitcher_cum_home_runs = pitcher.cum_home_runs(pitcher_against)      
        pitcher_cum_walks = pitcher.cum_walks(pitcher_against)
        pitcher_cum_outs = pitcher.cum_outs(pitcher_against)

        
        if(num<batter.singles(batter_against)):
            batter_cum_singles+=1
            pitcher_cum_singles+=1
            return "single"
        elif(num<batter.doubles(batter_against)):
            batter_cum_doubles+=1
            pitcher_cum_doubles+=1
            return "double"
        elif(num<batter.triples(batter_against)):
            batter_cum_triples+=1
            pitcher_cum_triples+=1
            return "triple"
        elif(num<batter.home_runs(batter_against)):
            batter_cum_home_runs+=1
            pitcher_cum_home_runs+=1
            return "home_run"
        elif(num<batter.walks(batter_against)):
            batter_cum_walks+=1
            pitcher_cum_walks+=1
            return "walk"
        elif(num<1):
            pitcher_cum_outs+=1
            return "out"
        elif(num<pitcher.singles(pitcher_against)+1):
            batter_cum_singles+=1
            pitcher_cum_singles+=1
            return "single"
        elif(num<pitcher.doubles(pitcher_against)+1):
            batter_cum_doubles+=1
            pitcher_cum_doubles+=1
            return "double"
        elif(num<pitcher.triples(pitcher_against)+1):
            batter_cum_triples+=1
            pitcher_cum_triples+=1
            return "triple"
        elif(num<pitcher.home_runs(pitcher_against)+1):
            batter_cum_home_runs+=1
            pitcher_cum_home_runs+=1
            return "home_run"
        elif(num<pitcher.walks(pitcher_against)+1):
            batter_cum_walks+=1
            pitcher_cum_walks+=1
            return "walk"
        else:
            pitcher_cum_outs+=1
            return "out"
        
    def move_runners(self, team, pitcher, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            pitcher: pitcher pitching to team
            result: str of result for batter (ie "walk")
        """
        if(result=="single"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.lineup[team.batting_index]

        elif(result=="double"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.lineup[team.batting_index]
            team.runners[0] = None

        elif(result=="triple"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.lineup[team.batting_index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="home_run"):
            team.lineup[team.batting_index].cum_runs+=1
            pitcher.cum_runs+=1
            pitcher.current_runs+=1
            team.lineup[team.batting_index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                pitcher.current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="walk"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.lineup[team.batting_index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        temp_2.cum_runs+=1
                        pitcher.cum_runs+=1
                        pitcher.current_runs+=1
                        team.lineup[team.batting_index].cum_rbi+=1
                        team.runners[3]+=1

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0  
            
    def print_result(self):
        """Printed score of a game""" 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams""" 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played""" 
        self.away_team.restart()
        self.away_team.game_no+=1
        away_team_rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        home_team_rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                outs = 0
                
                if(inning==1):
                    
                    if(side==self.away_team):
                        pitcher = self.home_team.rotation[home_team_rotation_index]
                        self.home_team.current_pitcher_list.append(pitcher)
                        self.home_team.current_pitcher = pitcher
                    elif(side==self.home_team):
                        pitcher = self.away_team.rotation[away_team_rotation_index]
                        self.away_team.current_pitcher_list.append(pitcher)
                        self.away_team.current_pitcher = pitcher

                while(outs<3):
                    if(side==self.away_team):
                        pitcher = self.home_team.choose_pitcher(outs, inning, side.runs, self.home_team.runs, 
                                                                side.lineup, side.batting_index)
                        print(pitcher.name + ' here')
                    elif(side==self.home_team):
                        pitcher = self.away_team.choose_pitcher(outs, inning, side.runs, self.away_team.runs, 
                                                                side.lineup, side.batting_index)
                        print(pitcher.name + ' here')

                        
                    stolen_base_num = random.uniform(0,1)
                    stolen_base_result = self.get_stolen_base_result(side.runners, pitcher, stolen_base_num)
                    
                    if(stolen_base_result=="out"):
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):
                            pitcher.finished_half = True
                            break
                                
                    result_num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.batting_index], pitcher, result_num)
                    
                    if(result=="out"):
                        pitcher.current_outs+=1
                        outs+=1
                    else:
                        self.move_runners(side, pitcher, result)
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):
                        break
                        
                    side.batting_index+=1
                    side.batting_index = side.batting_index%8
                    
                if(game_over):
                    break
                        
                side.reset_runners()
                
            if(game_over):
                break   
                
            inning+=1
        
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season_Improved(Game_Improved):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game_Improved(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers)
 
class FullSeason_Improved(Game_Improved):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, schedule):
        """Initializes values for this class"""
        self.schedule = schedule

    def simulate_season(self):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for date in self.schedule['date'].unique():
            for acronym, city in acronym_to_city_dict.items():
                away_team = team_dict_df[acronym_to_city_dict[acronym]]
                home_team_acronym = self.schedule.loc[self.schedule['date']==date][acronym].values[0]

                if(not pd.isna(home_team_acronym)):
                    home_team = team_dict_df[acronym_to_city_dict[home_team_acronym]]
                    g = Game_Improved(away_team, home_team)
                    g.play() 
  

    def reset_season(self):
        
        for city_name, team in team_dict_df.items():
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers)   
            
    def print_record(self):
        
        for city_name, team in team_dict_df.items():
            print(team.city + ': ' + str(team.wins) + "-" + str(team.losses))
                       

In [62]:
team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict, relievers_dict)
g = Game_Improved(team_dict_df['Atlanta'], team_dict_df['Washington'])
g.play()
g.print_result()

Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Max Scherzer here
Mike Soroka here
Mike Soroka here
Mike Soroka here
Max Scherzer here
Max Scherzer here
M

In [44]:
g.play()
g.print_result()

Stephen Strasburg here
Stephen Strasburg here
Stephen Strasburg here
2
Stephen Strasburg here
Mike Foltynewicz here
Mike Foltynewicz here
Mike Foltynewicz here
2
Mike Foltynewicz here
3
Stephen Strasburg here
6
Stephen Strasburg here
12
Stephen Strasburg here
15
Stephen Strasburg here
13
Stephen Strasburg here
22
Stephen Strasburg here
23
Stephen Strasburg here
3
Mike Foltynewicz here
7
Mike Foltynewicz here
5
Mike Foltynewicz here
21
Stephen Strasburg here
24
Stephen Strasburg here
25
Stephen Strasburg here
29
Stephen Strasburg here
6
Mike Foltynewicz here
7
Mike Foltynewicz here
11
Mike Foltynewicz here
30
Stephen Strasburg here
28
Stephen Strasburg here
32
Stephen Strasburg here
35
Stephen Strasburg here
35
Stephen Strasburg here
38
Stephen Strasburg here
12
Mike Foltynewicz here
10
Mike Foltynewicz here
11
Mike Foltynewicz here
39
Stephen Strasburg here
43
Stephen Strasburg here
44
Stephen Strasburg here
15
Mike Foltynewicz here
13
Mike Foltynewicz here
14
Mike Foltynewicz here
42


In [3]:
class Batter:
    """Class that holds information for a Batter.
    
    Attributes:
        team: team name of player
        name: player name
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks, sb_rate, sb_success):
        """Initializes values for this class"""
        
        self.team = team
        self.name = name
        self.singles = singles
        self.doubles = singles + doubles
        self.triples = singles + doubles + triples
        self.home_runs = singles + doubles + triples + home_runs
        self.walks = singles + doubles + triples + home_runs + walks
        self.sb_rate = sb_rate
        self.sb_success = sb_success
        self.cum_singles = 0
        self.cum_doubles = 0
        self.cum_triples = 0
        self.cum_home_runs = 0
        self.cum_walks = 0
        self.cum_pa = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
    
    def hits(self):
        """total hits of a batter
        
        Returns:
            int: number of hits
        """
        return(self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if(self.cum_pa-self.cum_walks>0):
            return(round(float((self.hits())/(self.cum_pa-self.cum_walks)), 3))
        else:
            return 0    
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa>0):
            return(round((self.hits()+self.cum_walks)/(self.cum_pa), 3))
        else:
            return 0
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form"""
        if(self.cum_pa-self.cum_walks>0):
            return(round((self.cum_singles+2*self.cum_doubles+3*self.cum_triples+4*self.cum_home_runs)/(self.cum_pa-self.cum_walks), 3))
        else:
            return 0
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg() + self.obp())
    def print_stats(self):
        """Prints some stats of a player"""

        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str(self.cum_home_runs))
        
class Pitcher(Batter):
    """Class that holds information for a pitcher.
    
    Attributes:
        team: team name of player
        name: player name
        position: player position
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks, sb, cs):
        """Initializes values for this class"""
        
        Batter.__init__(self, team, name, singles, doubles, triples, home_runs, walks, sb, cs)
        self.cum_outs = 0
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)"""
        ip = self.cum_outs/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form"""
        if(self.cum_outs==0):
            return 0
        else:
            return(round((27*self.cum_runs/self.cum_outs), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form"""
        if(self.cum_outs==0):
            return 0
        else:
            return(round(((self.cum_walks+self.hits())/(self.cum_outs/3)), 2))
    
class Team:
    """Class that holds information for a team.
    
    Attributes:
        city: team city
        roster: batting lineup of team in list form
        batting_index: position in batting order of current batter
        rotation_index: position in rotation of current pitcher
        runners: list representing whether a runner is on each base
        runs: number of runs fr a team in a game
        wins: number of wins for a team
        losses: number of losses for a team
        cumulative runs: total runs for a team over a span of games
    """
    def __init__(self, city, league, batter_df, pitcher_df, starters):
        """Initializes values for this class"""
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.pitcher_df = pitcher_df
        self.starters = starters
        self.pitching_staff = pitcher_df
        self.batting_index = 0
        self.rotation_index = 0
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        self.pitchers = []
        self.batters = []
        self.game_no = 0
        
        for index, batter in batter_df.iterrows():   # adding attributes to each batter and instantiating Batter class
            num_singles = batter['H']-batter['2B']-batter['3B']-batter['HR']
            pa = batter['AB'] + batter['BB']
            singles = round(num_singles/pa, 3)
            doubles = round(batter['2B']/pa, 3)
            triples = round(batter['3B']/pa, 3)
            home_runs = round(batter['HR']/pa, 3)
            walks = round(batter['BB']/pa, 3)
            sb_rate = round((batter['SB']+batter['CS'])/(num_singles+batter['BB']), 3)
            if(batter['CS'])==0:
                sb_success = 0
            else:
                sb_success = batter['SB']/batter['CS']

            self.batters.append(Batter(self.city, batter['PLAYER'], singles, doubles, triples, home_runs, walks, sb_rate, sb_success))

        for index, pitcher in pitcher_df.iterrows():    # adding attributes to each pitcher and instantiating Pitcher class
            ip = math.modf(pitcher['IP'])
            outs = int(ip[0]*10 + ip[1]*3)
            total_batters = outs + pitcher['BB'] + pitcher['H']
            hit_ratio = pitcher['H']/total_batters
            singles = round(hit_ratio*.64, 3)
            doubles = round(hit_ratio*.2, 3)
            triples = round(hit_ratio*.02, 3)
            home_runs = round(hit_ratio*.14, 3)
            walks = round(pitcher['BB']/total_batters, 3)

            self.pitchers.append(Pitcher(self.city, pitcher['PLAYER'], singles, doubles, triples, home_runs, walks, 0, 0))

        def make_lineup(batters, starters, league):
            lineup = []
            
            for starter in starters['lineup']:
                for batter in batters:
                    if(batter.name==starter):
                        lineup.append(batter)
                        
            if(league=='NL'):
                while(len(lineup)<8):
                    lineup.append(Batter(self.city, 'extra' +str(len(lineup)), .15, .05, .02, .03, .04, .02, .01))
                
                lineup.append(Batter(self.city, 'pitcher', .08, .02, .005, .01, .04, 0, 0))
            elif(league=='AL'):
                while(len(lineup)<9):
                    lineup.append(Batter(self.city, 'extra' +str(len(lineup)), .15, .05, .02, .03, .04, .02, .01))             
                
            return lineup
        
        def make_rotation(pitchers, starters):
            rotation = []
            
            for starter in starters['rotation']:
                for pitcher in pitchers:
                    if(pitcher.name==starter):
                        rotation.append(pitcher)
                        
            while(len(rotation)<5):
                rotation.append(Pitcher(self.city, 'extra'+ str(len(rotation)), .16, .06, .02, .03, .05, 0, 0))
                
            return rotation
        
        self.lineup = make_lineup(self.batters, self.starters, self.league)
        self.rotation = make_rotation(self.pitchers, self.starters)
        
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started"""
        self.batting_index = 0
        self.runners = [None, None, None, 0]
        self.runs = 0
        
    def reset_runners(self):
        """Resets baserunners after each half inning"""
        self.runners = [None, None, None, 0]
    
    def cum_stats_df(self):
        """Class that holds information for a Batter.
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats"""
        
        batter_df = pd.DataFrame(columns=['Name', 'PA', '1B', '2B', '3B', 'HR', 'BB', 'AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_pa, b.cum_singles, b.cum_doubles, b.cum_triples, b.cum_home_runs, b.cum_walks,
                             b.avg(), b.obp(), b.slg(), b.ops(), b.cum_runs, b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
     
        pitcher_df = pd.DataFrame(columns=['Name', 'IP', 'H', 'BB', 'ERA', 'WHIP', 'HR','AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'SB', 'CS'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.ip(), p.hits(), p.cum_walks, p.era(), p.whip(), p.cum_home_runs, p.avg(),
                             p.obp(), p.slg(), p.ops(), p.cum_runs, p.cum_sb, p.cum_cs]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
            
        return batter_df, pitcher_df
    
class Game:
    """Class that holds information for a Game.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team
                                
    def get_stolen_base_result(self, runners, pitcher, num):
        
        if(runners[0]!=None and runners[1]==None):
            if(num>runners[0].sb_rate):
                return None
            else:
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        else:
            return None
        
    def get_result(self, batter, pitcher, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """
        batter.cum_pa+=1
        pitcher.cum_pa+=1
        
        if(num<batter.singles):
            batter.cum_singles+=1
            pitcher.cum_singles+=1
            return "single"
        elif(num<batter.doubles):
            batter.cum_doubles+=1
            pitcher.cum_doubles+=1
            return "double"
        elif(num<batter.triples):
            batter.cum_triples+=1
            pitcher.cum_triples+=1
            return "triple"
        elif(num<batter.home_runs):
            batter.cum_home_runs+=1
            pitcher.cum_home_runs+=1
            return "home_run"
        elif(num<batter.walks):
            batter.cum_walks+=1
            pitcher.cum_walks+=1
            return "walk"
        elif(num<1):
            pitcher.cum_outs+=1
            return "out"
        elif(num<pitcher.singles+1):
            batter.cum_singles+=1
            pitcher.cum_singles+=1
            return "single"
        elif(num<pitcher.doubles+1):
            batter.cum_doubles+=1
            pitcher.cum_doubles+=1
            return "double"
        elif(num<pitcher.triples+1):
            batter.cum_triples+=1
            pitcher.cum_triples+=1
            return "triple"
        elif(num<pitcher.home_runs+1):
            batter.cum_home_runs+=1
            pitcher.cum_home_runs+=1
            return "home_run"
        elif(num<pitcher.walks+1):
            batter.cum_walks+=1
            pitcher.cum_walks+=1
            return "walk"
        else:
            pitcher.cum_outs+=1
            return "out"
        
    def move_runners(self, team, pitcher, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            pitcher: pitcher pitching to team
            result: str of result for batter (ie "walk")
        """
        if(result=="single"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.lineup[team.batting_index]

        elif(result=="double"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.lineup[team.batting_index]
            team.runners[0] = None

        elif(result=="triple"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.lineup[team.batting_index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="home_run"):
            team.lineup[team.batting_index].cum_runs+=1
            pitcher.cum_runs+=1
            team.lineup[team.batting_index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="walk"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.lineup[team.batting_index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        temp_2.cum_runs+=1
                        pitcher.cum_runs+=1
                        team.lineup[team.batting_index].cum_rbi+=1
                        team.runners[3]+=1

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0  
            
    def print_result(self):
        """Printed score of a game""" 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams""" 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played""" 
        self.away_team.restart()
        self.away_team.game_no+=1
        away_team_rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        home_team_rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                outs = 0
                
                if(side==self.away_team):
                    pitcher = self.home_team.rotation[home_team_rotation_index]
                elif(side==self.home_team):
                    pitcher = self.away_team.rotation[away_team_rotation_index]

                while(outs<3):
                    stolen_base_num = random.uniform(0,1)
                    stolen_base_result = self.get_stolen_base_result(side.runners, pitcher, stolen_base_num)
                    
                    if(stolen_base_result=="out"):
                        outs+=1
                                
                        if(outs>2):
                            break
                                
                    result_num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.batting_index], pitcher, result_num)
                    if(result=="out"):
                        outs+=1
                    else:
                        self.move_runners(side, pitcher, result)
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):
                        break
                        
                    side.batting_index+=1
                    side.batting_index = side.batting_index%8
                    
                if(game_over):
                    break
                        
                side.reset_runners()
                
            if(game_over):
                break   
                
            inning+=1
        
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters)
            
class FullSeason(Game):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, schedule):
        """Initializes values for this class"""
        self.schedule = schedule

    def simulate_season(self):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for date in self.schedule['date'].unique():
            for acronym, city in acronym_to_city_dict.items():
                away_team = team_dict_df[acronym_to_city_dict[acronym]]
                home_team_acronym = self.schedule.loc[self.schedule['date']==date][acronym].values[0]

                if(not pd.isna(home_team_acronym)):
                    home_team = team_dict_df[acronym_to_city_dict[home_team_acronym]]
                    g = Game(away_team, home_team)
                    g.play() 
  

    def reset_season(self):
        
        for city_name, team in team_dict_df.items():
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters)   
            
    def print_record(self):
        
        for city_name, team in team_dict_df.items():
            print(team.city + ': ' + str(team.wins) + "-" + str(team.losses))
                       

In [3]:
def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)
        
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
  
def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def column_type(batter_stats, pitcher_stats):
    string_columns = ['Season', 'Name', 'Bats', 'Tm']
    int_columns = ['playerId', 'G_R', 'PA_R', 'AB_R',
                   'H_R', '1B_R', '2B_R', '3B_R', 'HR_R', 'R_R', 'RBI_R', 'BB_R', 'IBB_R','SO_R', 'HBP_R', 'SF_R',
                   'SH_R', 'GDP_R', 'SB_R', 'CS_R', 'G_L', 'PA_L', 'AB_L', 'H_L', '1B_L', '2B_L', '3B_L', 'HR_L', 
                   'R_L', 'RBI_L', 'BB_L', 'IBB_L', 'SO_L', 'HBP_L', 'SF_L', 'SH_L', 'GDP_L', 'SB_L', 'CS_L']
    float_columns = ['AVG_R', 'AVG_L']

    for column in batter_stats.columns:
        if(column in int_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_float(row, column), axis=1)

    string_columns = ['Season', 'Name', 'Throws', 'Team']
    int_columns = ['playerId', 'IP_L', 'TBF_L', 'H_L', '2B_L', '3B_L', 'R_L', 'ER_L', 'HR_L', 'BB_L', 'IBB_L', 'HBP_L',
                   'SO_L', 'IP_R', 'TBF_R', 'H_R', '2B_R', '3B_R', 'R_R', 'ER_R', 'HR_R', 'BB_R', 'IBB_R', 'HBP_R','SO_R']
    float_columns = ['ERA_L', 'AVG_L', 'OBP_L', 'SLG_L', 'wOBA_L', 'ERA_R', 'AVG_R', 'OBP_R', 'SLG_R', 'wOBA_R']


    for column in pitcher_stats.columns:
        if(column in int_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_float(row, column), axis=1)     
            
    return batter_stats, pitcher_stats
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('-')
        day = split_date[0]
        day = day.zfill(2)
        month = strptime(split_date[1],'%b').tm_mon
        date = str(month) +  '/' + day + '/' + '2020'
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')
        
    return date

def split_stats():   
    url = 'http://www.espn.com/mlb/player/splits/_/id/31283/type/batting3'

    headers = {'User-Agent': header_name}

    platoon_dict = {}
    data_columns = ['Split', 'Year', 'Tm', 'team_ID', 'Pos', 'POS', 'pos_season', 'Name', 'player', 'Age', 'age', 'G', 'PA', 'AB', 
                    'R', 'H', '2B', '3B', 'HR', 'SB', 'CS', 'BB', 'SO', 'HBP']



    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    left_columns = ['Name', 'Bats']
    right_columns = []
    name = soup.h1.get_text()

    for bio in soup.findAll('div', attrs={'class': 'player-bio'}):
        for info in bio.findAll('ul', attrs={'class': 'general-info'}):
            bats = info.get_text().split('Bats')[1][2]

    for table in soup.findAll('table', attrs={'class': 'tablehead'}):
        labels = table.find('tr', attrs={'class': 'colhead'})

        for header_value in labels.findAll('td'):
            left_columns.append(header_value.get_text()+'L')
            right_columns.append(header_value.get_text()+'R')

        df = pd.DataFrame(columns=(left_columns+right_columns))            
        player_values = [name, bats]

        for column_value in table.findAll('tr', attrs={'class': ['oddrow', 'evenrow']}):
            if(column_value.td.get_text() in ['vs. Left', 'vs. Right']):
                for value in column_value.findAll('td'):
                    player_values.append(value.get_text())

        df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

    df.drop(['OverallL', 'OverallR'], axis=1, inplace=True)   
    print(df)

#df['date'] = df.apply(lambda row: date_formatter(row), axis=1)
#print(df)
    
#writing_data(batter_stats_df, 'mlb_batter_stats_2019.csv')
#writing_data(pitcher_stats_df, 'mlb_pitcher_stats_2019.csv')        

In [4]:
def batter_stats():

    url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
    links = [url+str(i) for i in range(1, 350, 50)]
    headers = {'User-Agent': header_name}

    batter_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            batter_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        batter_stats = pd.concat([batter_stats, df], ignore_index=True)

    return batter_stats

def pitcher_stats():
    url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
    links = [url+str(i) for i in range(1, 500, 50)]
    headers = {'User-Agent': header_name}

    pitcher_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            pitcher_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

    return pitcher_stats

def projected_starters():

    url = 'https://www.mlb.com/news/projecting-every-mlb-lineup-rotation'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    starters = {}

    for body in soup.findAll('div', attrs={'class': 'article-item__body'}):
        pitchers = []
        batters = []
        closer = None

        for team in body.findAll('p'):
            if(team.strong and team.strong.get_text().lower() in nickname_to_city_dict.keys()):
                current_team = team.strong.get_text().lower()
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    batters.append(unidecode.unidecode(entry.lstrip()))
                
            elif(team.strong and team.strong.get_text()=='Pitchers'):
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    pitchers.append(unidecode.unidecode(entry.lstrip()))
            elif(team.strong and team.strong.get_text()=='Closer:'):
                closer = unidecode.unidecode(team.find(['forge-entity', 'a']).get_text())
            else:
                continue

            if(closer!=None):
                starters.update({nickname_to_city_dict[current_team]: 
                                {'lineup': batters, 'rotation': pitchers, 'closer': closer}})
                pitchers = []
                batters = []
                closer = None
                
    return starters

def projected_relievers():
    league_relief_dict = {}
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='

    for num in range(1,31):
        url_list.append(url_start+str(num))

    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        relief_df = pd.DataFrame(columns=['Name', 'IP'])


        for a_tag in soup.findAll('a', attrs={'href': '#RP'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')
            relief_table = a_tag.find_next('table')

            for row in relief_table.findAll('tr', attrs={'class': 'depth_reg'}):
                name_tag = row.find_next('td')
                name = name_tag.get_text()
                ip = name_tag.find_next('td').get_text()
                relief_df = relief_df.append({'Name': name, 'IP': ip}, ignore_index=True)

        league_relief_dict.update({nickname_to_city_dict[str(team_name.lower())]: relief_df})
        
    return league_relief_dict

def create_teams(batter_stats, pitcher_stats, starters, relievers):
    url_dict = {}
    url_start = 'https://www.espn.com/mlb/team/roster/_/name/'
    team_dict = {}
    

    for acronym, city in acronym_to_city_dict.items():
        url_dict.update({url_start+acronym.lower(): city})

    for url, city in url_dict.items():
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        player_id_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for section in soup.findAll('section'):
            for player in section.findAll('td', attrs={'class': 'Table__TD'}):
                row = player.find('a', attrs={'class': 'AnchorLink'})
                pos = player.find('span')

                if(row and row.get_text()!=''):
                    if(section.div.get_text()=='Pitchers'):
                        player_id_dict.update({row.get_text(): {int(row['href'].split('/')[-1]): 'P'}})
                    else:
                        player_id_dict.update({row.get_text(): {int(row['href'].split('/')[-1]): 'H'}})
                   
        for name, df in player_id_dict.items():
            for ID, pos in df.items():
                if(pos=='P'):
                    if(name in pitcher_stats['Name'].tolist()):
                        stats = pitcher_stats.loc[pitcher_stats['Name']==name]
                        col_headers = stats.columns
                    else:
                        stats = pd.Series({'Season': 2019, 'Name': name, 'playerId': 1234, 'Team': city, 'Throws': 'R', 
                                           'IP_L': 30, 'TBF_L': 130, 'ERA_L': 5.40, 'H_L': 30, '2B_L': 5, 
                                           '3B_L': 1, 'R_L': 18, 'ER_L': 18, 'HR_L': 4, 'BB_L': 17, 'IBB_L': 0,  
                                           'HBP_L': 3, 'SO_L': 50, 'AVG_L': .250, 'OBP_L': .385, 'SLG_L': .544, 'wOBA_L': .370, 
                                           'IP_R': 30, 'TBF_R': 130, 'ERA_R': 5.40, 'H_R': 30, '2B_R': 5, 
                                           '3B_R': 1, 'R_R': 18, 'ER_R': 18, 'HR_R': 4, 'BB_R': 17, 'IBB_R': 0,  
                                           'HBP_R': 3, 'SO_R': 50, 'AVG_R': .250, 'OBP_R': .385, 'SLG_R': .544, 'wOBA_R': .370})
                        
                        col_headers = stats.index
                    if(pitcher_df.empty):
                        pitcher_df = pd.DataFrame(columns=col_headers)
                    
                    pitcher_df = pitcher_df.append(stats, ignore_index=True)

                elif(pos=='H'):
                    if(name in batter_stats['Name'].tolist()):
                        stats = batter_stats.loc[batter_stats['Name']==name]
                        col_headers = stats.columns
                    else:
                        stats = pd.Series({'Season': 2019, 'Name': name, 'playerId': 1234, 'Team': city, 'Bats': 'R', 
                                           'G_L': 60, 'PA_L': 220, 'AB_L': 200, 'H_L': 50, '1B_L': 34, '2B_L': 10, 
                                           '3B_L': 1, 'HR_L': 5, 'R_L': 25, 'RBI_L': 25, 'BB_L': 20, 'IBB_L': 0, 'SO_L': 50, 
                                           'HBP_L': 3, 'SF_L': 2, 'SH_L': 0, 'GDP_L': 3, 'SB_L': 5, 'CS_L': 1, 'AVG_L': .250, 
                                           'G_L': 60, 'PA_R': 220, 'AB_R': 200, 'H_R': 50, '1B_R': 34, '2B_R': 10, '3B_R': 1,
                                           'HR_R': 5, 'R_R': 25, 'RBI_R': 25, 'BB_R': 20, 'IBB_R': 0, 'SO_R': 50, 'HBP_R': 3,
                                           'SF_R': 2, 'SH_R': 0, 'GDP_R': 3, 'SB_R': 5, 'CS_R': 1, 'AVG_R': .250})
                        
                        col_headers = stats.index

                    if(batter_df.empty):
                        batter_df = pd.DataFrame(columns=col_headers)

                    batter_df = batter_df.append(stats, ignore_index=True)

        team_dict.update({city: Team_Improved(city, league[city], batter_df, pitcher_df, starters[city], relievers[city])})
        
    return team_dict

In [19]:
url_start = 'https://www.fangraphs.com/teams/'
url_dict = {}
b_dict = {}
p_dict = {}
for nickname, city in nickname_to_city_dict.items():
    url_dict.update({url_start+nickname: city})

for url, city in url_dict.items():
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    batter_list = []
    pitcher_list = []
    
    for num, table in enumerate(soup.findAll('div', attrs={'class': 'team-stats-table'})):
        for player in table.findAll('td', attrs={'class': 'frozen'}):
            if(num==0 and player.get_text()!='Team Total'):
                batter_list.append(player.get_text())  
            if(num==1 and player.get_text()!='Team Total'):
                pitcher_list.append(player.get_text()) 
                
    b_dict.update({city: batter_list})
    p_dict.update({city: pitcher_list})

In [5]:
relievers_dict = projected_relievers()

In [6]:
batter_stats_df = reading_data('Splits_Leaderboard_Data_Hitters.csv')
pitcher_stats_df = reading_data('Splits_Leaderboard_Data_Pitchers.csv')
batter_stats_df, pitcher_stats_df = column_type(batter_stats_df, pitcher_stats_df)
starters_dict = projected_starters()
schedule = reading_data('2020_mlb_schedule.csv')
team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict, relievers_dict)

In [155]:
s = Season_Improved(team_dict_df['Detroit'], team_dict_df['Washington'])
s.reset_season([team_dict_df['Detroit'], team_dict_df['Washington']])
s.simulate_season(162)
s.print_record()

72-90
90-72


In [7]:
s = FullSeason_Improved(schedule)
s.reset_season()
s.simulate_season()
s.print_record()

KeyError: 'NYM'

In [116]:
for acronym, city in acronym_to_city_dict.items():
    away_team = team_dict_df[acronym_to_city_dict[acronym]]
    home_team_acronym = schedule.loc[schedule['date']=='3/28/2020'][acronym].values[0]

    if(not pd.isna(home_team_acronym)):
        home_team = team_dict_df[acronym_to_city_dict[home_team_acronym]]
        g = Game(away_team, home_team)
        g.play() 
        g.print_result()

final score: New York Mets 2 Washington 4
final score: Miami 3 Philadelphia 1
final score: Milwaukee 6 Chicago Cubs 3
final score: Cincinatti 0 St. Louis 5
final score: Los Angeles Dodgers 5 San Francisco 4
final score: Arizona 1 Atlanta 3
final score: San Diego 4 Colorado 2
final score: Tampa Bay 0 Pittsburg 4
final score: Toronto 3 Boston 6
final score: Baltimore 2 New York Yankees 1
final score: Cleveland 1 Detroit 3
final score: Chicago White Sox 10 Kansas City 1
final score: Houston 5 Los Angeles Angels 3
final score: Oakland 4 Minnesota 2
final score: Seattle 3 Texas 2


In [41]:
for p in team_dict_df['New York Yankees'].lineup:
    print(p.singles_L)

NameError: name 'team_dict_df' is not defined

In [74]:
league_relief_dict = {}
url_list = []
url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    
for num in range(1,31):
    url_list.append(url_start+str(num))
    
for url in url_list:
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    relief_df = pd.DataFrame(columns=['Name', 'IP'])


    for a_tag in soup.findAll('a', attrs={'href': '#RP'}):
        team_name = a_tag.find_next('span').get_text()
        team_name = team_name.replace('\xa0', '')
        relief_table = a_tag.find_next('table')

        for row in relief_table.findAll('tr', attrs={'class': 'depth_reg'}):
            name_tag = row.find_next('td')
            name = name_tag.get_text()
            ip = name_tag.find_next('td').get_text()
            relief_df = relief_df.append({'Name': name, 'IP': ip}, ignore_index=True)

    league_relief_dict.update({nickname_to_city_dict[str(team_name.lower())]: relief_df})

In [40]:
league_relief_dict['Atlanta']

NameError: name 'league_relief_dict' is not defined

In [68]:
starters_dict['Houston']

{'lineup': ['George Springer',
  'Alex Bregman',
  'Jose Altuve',
  'Carlos Correa',
  'Yordan Alvarez',
  'Yuli Gurriel',
  'Michael Brantley',
  'Josh Reddick',
  'Martin Maldonado'],
 'rotation': ['Justin Verlander',
  'Zack Greinke',
  'Lance McCullers Jr.',
  'Jose Urquidy',
  'Brad Peacock'],
 'closer': 'Roberto Osuna'}